##Stage 0: Install Dependencies

In [ ]:
!pip install ipython h5py nltk joblib jupyter pandas scipy


In [ ]:
!pip install ninja yacs>=0.1.8 cython matplotlib tqdm opencv-python numpy>=1.19.5

In [ ]:
!pip install timm einops

In [ ]:
!pip install cityscapesscripts

In [ ]:
!pip install pycocotools

In [ ]:
!pip install yamlloader

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
#!git clone https://github.com/microsoft/scene_graph_benchmark

In [ ]:
%cd scene_graph_benchmark

In [ ]:
!python setup.py build develop

##Extract Features Using VinVl

##Modify the /content/drive/MyDrive/scene_graph_benchmark/sgg_configs/vgattr/vinvl_x152c4.yaml file before running VinVl

There are four entries to be changed: 

1. **DATASET.TEST**: this is an entry of the yaml file denoting the paths of the tsv files for VinVl input. This yaml file should be stored in the data directory (which corresponds to **DATA_DIR**). This yaml file is used to build up ***VGTSVDataset*** (scene_graph_benchmark/maskrcnn_benchmark/data/datasets/vg_tsv.py) 
for dataloader.                                                         Only images tsv file need to be present in this yaml files, as other three tsv files, label_file, hw_file, linelist_file, are optinal for building the super ***TSVDataset*** (scene_graph_benchmark/maskrcnn_benchmark/data/datasets/tsv_dataset.py).                                                                       This is indicated by line 5 of the ***config_tsv_dataset_args*** in scene_graph_benchmark/maskrcnn_benchmark/data/datasets/utils/config_args.py; line 30, 40, 42, and 170 in scene_graph_benchmark/maskrcnn_benchmark/data/build.py; line 55 of scene_graph_benchmark/tools/test_sg_net.py.

2. **DATA_DIR**: this is an entry of the yaml file denoting the directory that stores tsv yaml files, as indicated by line 5 of maskrcnn_benchmark/data/datasets/utils/config_args.py. 

3. **DATASETS.LABELMAP_FILE**: this is the entry that contains the full path of the label map json file, as indicated by line 56 of scene_graph_benchmark/tools/test_sg_net.py and line 30 of maskrcnn_benchmark/data/datasets/utils/load_files.py.

4. **OUTPUT_DIR**: this is the entry that denotes the directory where the result of VinVl inference will be stored.  


##Fetch Image Files according to Image Names

In [ ]:
# visualize VinVL object detection
# pretrained models at https://penzhanwu2.blob.core.windows.net/sgg/sgg_benchmark/vinvl_model_zoo/vinvl_vg_x152c4.pth
# the associated labelmap at https://penzhanwu2.blob.core.windows.net/sgg/sgg_benchmark/vinvl_model_zoo/VG-SGG-dicts-vgoi6-clipped.json

import cv2
import os
import os.path as op
import argparse
import json
from PIL import Image
import numpy as np


from scene_graph_benchmark.scene_parser import SceneParser
from scene_graph_benchmark.AttrRCNN import AttrRCNN
from maskrcnn_benchmark.data.transforms import build_transforms
from maskrcnn_benchmark.utils.checkpoint import DetectronCheckpointer
from maskrcnn_benchmark.config import cfg
from scene_graph_benchmark.config import sg_cfg
from maskrcnn_benchmark.data.datasets.utils.load_files import \
    config_dataset_file
from maskrcnn_benchmark.data.datasets.utils.load_files import load_labelmap_file
from maskrcnn_benchmark.utils.miscellaneous import mkdir


In [ ]:
from PIL import Image
import os
import pandas as pd

def cv2Img_to_Image(input_img):
    cv2_img = input_img.copy()
    img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    return img


def detect_objects_on_single_image(model, transforms, cv2_img):
    # cv2_img is the original input, so we can get the height and 
    # width information to scale the output boxes.
    img_input = cv2Img_to_Image(cv2_img)
    img_input, _ = transforms(img_input, target=None)
    img_input = img_input.to(model.device)

    with torch.no_grad():
        prediction = model(img_input)[0].to('cpu')
    #     prediction = prediction[0].to(torch.device("cpu"))

    img_height = cv2_img.shape[0]
    img_width = cv2_img.shape[1]

    prediction = prediction.resize((img_width, img_height))
    
    return prediction

#Setting configuration
cfg.set_new_allowed(True)
cfg.merge_from_other_cfg(sg_cfg)
cfg.set_new_allowed(False)
#Configuring VinVl
cfg.merge_from_file('path_to_vinvl_x152c4.yaml')

#This is a list specifying the values for additional arguments, it encompasses pairs of list and values in an ordered manner
#MODEL.WEIGHT specifies the full path of the VinVl weight pth file
#DATA_DIR specifies the directory that contains VinVl input tsv configuration yaml file
argument_list = [
                 'MODEL.WEIGHT', 'path_to_vg_x152c4.pth',
                 'MODEL.ROI_HEADS.NMS_FILTER', 1,
                 'MODEL.ROI_HEADS.SCORE_THRESH', 0.2, 
                 'DATA_DIR', "",
                 'TEST.IGNORE_BOX_REGRESSION', False,
                 'MODEL.ATTRIBUTE_ON', True
                 ]
cfg.merge_from_list(argument_list)
cfg.freeze()

#     assert op.isfile(args.img_file), \
#         "Image: {} does not exist".format(args.img_file)

output_dir = cfg.OUTPUT_DIR
#     mkdir(output_dir)


model = AttrRCNN(cfg)
model.to(cfg.MODEL.DEVICE)
model.eval()

checkpointer = DetectronCheckpointer(cfg, model, save_dir=output_dir)
checkpointer.load(cfg.MODEL.WEIGHT)

# dataset labelmap is used to convert the prediction to class labels
# dataset_labelmap_file = config_dataset_file(cfg.DATA_DIR, cfg.DATASETS.LABELMAP_FILE)
#     assert dataset_labelmap_file
#path to input labelmap file
# labelmap_file = 'path_to_vinvl_labelmap.json'

# dataset_allmap = json.load(open(labelmap_file, 'r'))
# dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}

transforms = build_transforms(cfg, is_train=False)


  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import os
import PIL
import torch

dets = {}
greater_dataset_path = 'path_to_image_datasets'

def save_vinvl_prediction(train_csv_dataset_path, i):
# input_img_directory = 'path_to_images'
#need to be pth
  dets = {}

  train_dataset = pd.read_csv(train_csv_dataset_path) 

  train_data_names = list(train_dataset['img'])
  n = 0
  for img_name in train_data_names:
    if n%100 == 0:
      print(n)

    n=n+1

    img_file_path = os.path.join(greater_dataset_path, img_name)

    cv2_img = cv2.imread(img_file_path)

    img_height = cv2_img.shape[0]
    img_width = cv2_img.shape[1]

    det = detect_objects_on_single_image(model, transforms, cv2_img)
  
#   prediction contains ['labels',
#  'scores',
#  'box_features',
#  'scores_all',
#  'boxes_all',
#  'attr_labels',
#  'attr_scores']
# box_features are used for oscar


    det_dict ={key : det.get_field(key) for key in det.fields()}
    boxes = det.get_field('boxes_all')[:, 0,:]
    box_width = boxes[:, 2] - boxes[:, 0]
    box_height = boxes[:, 3] - boxes[:, 1]


    scaled_width = box_width / img_width
    scaled_height = box_height / img_height

    scaled_width = scaled_width[..., np.newaxis]
    scaled_height = scaled_height[..., np.newaxis]

    scaled_x = boxes[:, 0] / img_width
    scaled_y = boxes[:, 1] / img_width

    scaled_x = scaled_x[..., np.newaxis]
    scaled_y = scaled_y[..., np.newaxis]

    spatial_features = np.concatenate( (scaled_x, scaled_y, scaled_x + scaled_width, scaled_y + scaled_height, scaled_width, scaled_height), axis=1)
    det_dict['spatial_features'] = torch.tensor(spatial_features)
  


    dets[os.path.basename(img_name).split('.')[0]] = det_dict

    output_prediction_file_path = 'path_to_vinvl_output_'+str(i)+'.pth'
  torch.save(dets, output_prediction_file_path)
  
for i in range(0,5):
  save_vinvl_prediction('train_df'+str(i)+".csv", i)

In [ ]:
import json
labelmap_file = 'path_to_vinvl_labelmap.json'

dataset_allmap = json.load(open(labelmap_file, 'r'))
dataset_labelmap = {int(val): key for key, val in dataset_allmap['label_to_idx'].items()}

In [ ]:
import torch
import pandas as pd

def append_hateful(vinvl_output_path, train_df_path):
  vinvl_output_dict = torch.load(vinvl_output_path)
  train_df = pd.read_csv(train_df_path)
  dict_ids = list(vinvl_output_dict.keys())
  n = 0
  for id in dict_ids:
    if n%100==0:
      print(n)
    n+=1
    hateful = train_df.loc[train_df['id']==int(id)]['label'].values[0]
    vinvl_output_dict[id]['hateful'] = hateful
  torch.save(vinvl_output_dict, vinvl_output_path)

for i in range(0, 6):
  vinvl_output_path = 'path_to_vinvl_output_'+str(i)+'.pth'
  train_df_path = 'path_to_train_df_'+str(i)+'.csv'
  print(vinvl_output_path)
  print(train_df_path)
  append_hateful(vinvl_output_path, train_df_path)
  print("*****************************************")
  print()
  print()




In [ ]:
import torch
def clean_vinvl_output(vinvl_output_path):
  vinvl_output = torch.load(vinvl_output_path)
  dict_ids = list(vinvl_output.keys())
  n = 0
  for id in dict_ids:
    if n%100==0:
      print(n)
    n += 1
    new_entry = {}
    old_entry = vinvl_output[id]
    new_entry['labels'] = old_entry['labels']
    new_entry['box_features'] = old_entry['box_features']
    new_entry['caption'] = old_entry['caption']
    new_entry['hateful'] = old_entry['hateful']
    new_entry['spatial_features'] = old_entry['spatial_features']
    vinvl_output[id] = new_entry
  torch.save(vinvl_output, vinvl_output_path)

In [ ]:
for i in range(0, 6):
  vinvl_output_path = 'path_to_vinvl_output_'+str(i)+'.pth'
  print('********************************')
  print(vinvl_output_path)
  clean_vinvl_output(vinvl_output_path)
  print()
  print()

In [ ]:
%cd path_to_image_datasets

/content/drive/MyDrive/hateful_messages_dataset/image_datasets


In [ ]:
import pandas as pd
import os
import PIL
import torch
import json

dets = {}
greater_dataset_path = ''
jsonl_file_path = ''
greater_output_directory = ''
def vinvl_output_from_jsonl(jsonl_file_path, greater_dataset_path, greater_output_directory):

  with open(jsonl_file_path, 'r') as json_file:
    json_list = list(json_file)
  n = 0
  dets = {}
  for json_str in json_list:
    json_dict = json.loads(json_str)
    img_caption = json_dict['text']
    img_hateful = json_dict['label']
    img_name = os.path.basename(json_dict['img'])
    if n%100 == 0:
      print(n)
    n += 1

    img_file_path = os.path.join(greater_dataset_path, img_name)

    cv2_img = cv2.imread(img_file_path)

    img_height = cv2_img.shape[0]
    img_width = cv2_img.shape[1]

    det = detect_objects_on_single_image(model, transforms, cv2_img)
  
#   prediction contains ['labels',
#  'scores',
#  'box_features',
#  'scores_all',
#  'boxes_all',
#  'attr_labels',
#  'attr_scores']
# box_features are used for oscar

    necessary_fields = ['labels','box_features']
    det_dict ={key : det.get_field(key) for key in necessary_fields}
    boxes = det.get_field('boxes_all')[:, 0,:]
    box_width = boxes[:, 2] - boxes[:, 0]
    box_height = boxes[:, 3] - boxes[:, 1]


    scaled_width = box_width / img_width
    scaled_height = box_height / img_height

    scaled_width = scaled_width[..., np.newaxis]
    scaled_height = scaled_height[..., np.newaxis]

    scaled_x = boxes[:, 0] / img_width
    scaled_y = boxes[:, 1] / img_width

    scaled_x = scaled_x[..., np.newaxis]
    scaled_y = scaled_y[..., np.newaxis]

    spatial_features = np.concatenate( (scaled_x, scaled_y, scaled_x + scaled_width, scaled_y + scaled_height, scaled_width, scaled_height), axis=1)
    det_dict['spatial_features'] = torch.tensor(spatial_features)
    det_dict['caption'] = img_caption
    det_dict['hateful'] = img_hateful


    dets[os.path.basename(img_name).split('.')[0]] = det_dict

  output_prediction_file_path = os.path.join(greater_output_directory, 'path_to_test_vinvl_output.pth')
  torch.save(dets, output_prediction_file_path)
  
vinvl_output_from_jsonl(jsonl_file_path, greater_dataset_path, greater_output_directory)
